In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v3_small
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [2]:
data_dir = "FINAL_TRAINING_DATA"

In [3]:
transform = transforms.Compose([
    # transforms.Resize((224, 224)),  # this is original shape for MobileNet_v2
    transforms.Resize((128, 128)),  # for faster inference if accuracy is sufficient
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [4]:
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes
class_names

['DROWSY_NOT', 'DROWSY_YES']

In [5]:
dataset.class_to_idx

{'DROWSY_NOT': 0, 'DROWSY_YES': 1}

In [6]:
# Split into train (70%), val (15%), test (15%)
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

In [7]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [8]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
# pretrained MobileNet_v3_small model
model = mobilenet_v3_small(weights=True)
model = model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

In [13]:
for name, layer in model.named_modules():
    print(name, '→', layer)

 → MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1

In [16]:
# this is our final layer to be replaced
model.classifier[3]

Linear(in_features=1024, out_features=1000, bias=True)

In [17]:
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 1)
model.classifier[3].requires_grad = True  # only this new layer is trainable everything else is frozen

In [19]:
# verify which parameters are trainable; just double checking
for name, param in model.named_parameters():
    print(f"{name:60}  trainable={param.requires_grad}")

features.0.0.weight                                           trainable=False
features.0.1.weight                                           trainable=False
features.0.1.bias                                             trainable=False
features.1.block.0.0.weight                                   trainable=False
features.1.block.0.1.weight                                   trainable=False
features.1.block.0.1.bias                                     trainable=False
features.1.block.1.fc1.weight                                 trainable=False
features.1.block.1.fc1.bias                                   trainable=False
features.1.block.1.fc2.weight                                 trainable=False
features.1.block.1.fc2.bias                                   trainable=False
features.1.block.2.0.weight                                   trainable=False
features.1.block.2.1.weight                                   trainable=False
features.1.block.2.1.bias                                     tr

<b> Perfect. Only the last linear layer is active and all of the other layers are frozen.

In [21]:
criterion = nn.BCEWithLogitsLoss()  # BCE loss for binary classification
optimizer = optim.Adam(model.classifier[3].parameters(), lr=0.001)  # Optimize only the classifier head
num_epochs = 25

In [22]:
best_val_loss = float('inf')
patience = 6
counter = 0

for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
    train_loss /= len(train_loader)
    train_acc = correct / total

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    
    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1:02d}/{num_epochs}] "
          f"| Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:6.2f}% "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:6.2f}%")

    # ---- EARLY STOPPING ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")  # save best model
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

: 